# Organización de otros datos poblacionales

## Limpieza de los datos

<b>Objetivo:</b> eliminar las líneas innecesarias, y organizar los encabezados de los documentos. De esta manera podremos unificar todos los datos posteriormente.

In [1]:
import os

def clean_data(input_folder, output_folder):
    # Crear la carpeta de salida si no existe
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # Obtener la lista de archivos CSV en la carpeta de entrada
    csv_files = [archivo for archivo in os.listdir(input_folder) if archivo.endswith('.csv')]

    # Datos excepcionales
    unuseful_words = ['Censo', 'datos', 'Serie;;;']
    headers = f"Serie;Codigo_municipio;Nombre;2022;2023\n"

    # Limpiar y guardar los archivos CSV en la carpeta de salida
    for file in csv_files:
        original_file = os.path.join(input_folder, file)
        clean_file = os.path.join(output_folder, file)
        with open(original_file, 'r') as f_input, open(clean_file, 'w') as f_output:
            f_output.write(headers)
            delete_next = False
            for line in f_input:                    
                if not delete_next:
                    delete = False
                    for word in unuseful_words:
                        if word in line:
                            delete = True
                    if line.strip() == '':
                        delete = False
                    if not delete:
                        fields = line.strip().split(';')
                        f_output.write(';'.join(fields[:3] + fields[-3:]) + '\n')

                    # Eliminar líneas posteriores al último municipio
                    if 'Zarzalejo' in line:
                        delete_next = True


clean_data('./original_data', './clean_data')

## Densidad de población 2020

In [1]:
import os

def clean_data(input_file, output_file):

    # Datos excepcionales
    unuseful_words = ['Censo', 'datos', 'Serie;;;']
    headers = f"Serie;Codigo_municipio;Nombre;densidad_poblacion\n"

    with open(input_file, 'r') as f_input, open(output_file, 'w') as f_output:
        f_output.write(headers)
        delete_next = False
        for line in f_input:                    
            if not delete_next:
                delete = False
                for word in unuseful_words:
                    if word in line:
                        delete = True
                if line.strip() == '':
                    delete = False
                if not delete:
                    fields = line.strip().split(';')
                    f_output.write(';'.join(fields[:3] + fields[-5:-4]) + '\n')

                # Eliminar líneas posteriores al último municipio
                if 'Zarzalejo' in line:
                    delete_next = True


clean_data('./original_data/densidad_poblacion.csv', '../union_2020/densidad_poblacion.csv')

## Unión de los datos

<b>Objetivo:</b> unir todos los archivos de la carpeta en uno solo, con sus encabezados correspondientes. Guardar el <i>.csv</i> resultante en la carpeta <i>union_final</i> para poder añadir los datos al archivo final.

In [3]:
import os, csv

def join_data(folder_path, output_path):

    # Crear un diccionario para almacenar los datos unidos
    join_data = {}

    # Obtener la lista de archivos en la carpeta
    csv_files = os.listdir(folder_path)

    # Iterar sobre cada archivo en la carpeta
    for file in csv_files:
        if file.endswith('.csv'):
            with open(os.path.join(folder_path, file), 'r') as csv_file:
                csv_reader = csv.reader(csv_file, delimiter=';')
                # Saltar la primera fila que contiene los encabezados
                next(csv_reader)
                type = file.split('.')[0]
                    
                # Iterar sobre cada fila en el archivo CSV
                for row in csv_reader:
                    serie = row[0] if row else ""
                    codigo_municipio = row[1] if len(row) > 1 else ""
                    nombre = row[2] if len(row) > 2 else ""
                    
                    # Verificar si la serie, código de municipio y nombre ya están en el diccionario
                    if (serie, codigo_municipio, nombre) not in join_data:
                        join_data[(serie, codigo_municipio, nombre)] = {}

                    # Obtener los datos de cada año para el intervalo de edad actual
                    for i, valor in enumerate(row[3:5], start=2022):
                        clave = f"{type}_{i}"
                        join_data[(serie, codigo_municipio ,nombre)][clave] = valor

    # Escribir los datos unidos en un nuevo archivo CSV
    with open(output_path, 'w', newline='') as csv_file:
        csv_writer = csv.writer(csv_file, delimiter=';')
        final_keys = []

        for key, values in join_data.items():
            if final_keys == []:
                final_keys.extend(values.keys())
        final_keys = [str(clave) for clave in final_keys]
        csv_writer.writerow(['Serie', 'Codigo_municipio', 'Nombre', *final_keys])

        # Escribir los datos unidos
        for key, values in join_data.items():
            csv_writer.writerow([key[0], key[1], key[2], *values.values()])

join_data('./clean_data', '../union_final/datos_poblacionales.csv')